MS peak align based on bioAssay peak alignment for excipient screening
see nb-dejejord-5094579-001

In [76]:
#Loading Modules
import pandas as pd #handling of csv files and dataframes
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import streamlit as st


In [ ]:
st.set_page_config(page_title="Peak Alignment App", layout="wide")
st.title("LC Peak Alignment Tool")
st.markdown(
    "Hackathon Team Members: Kabir Dhingra, David Gray, Stephanie Mozley, Andrew Sinegra, and Kevin Wang "
)
st.markdown("---")

In [ ]:
# File uploader widget
uploaded_file = st.file_uploader("Upload an excel file of the peaks", type=["xlsx"])

# Check if a file has been uploaded
if uploaded_file is not None:
    # Read the uploaded file as a DataFrame
    data = pd.read_excel(uploaded_file)
    
    # Display the uploaded DataFrame
    st.write("Uploaded DataFrame:")
    st.write(data)

pre-processing and qc steps
return parent peak retention time median, min RT, max RT, enter global parent peak RT (optional)
parent API peak cutoff


In [78]:
df_vial_counts = data.groupby(['Vial']).size().reset_index(name='peakCount')
maxarea = data.groupby(['Vial'])['Area'].max().reset_index()
areasum = data.groupby(['Vial'])['Area'].sum().reset_index(name='Sum of Area')
maxarea['Area Sum'] = areasum['Sum of Area']
maxarea['Area Ratio'] = maxarea['Area']/ maxarea['Area Sum']*100
RTmax= (data.sort_values(['Vial', 'Area'], ascending=[True, False])
             .drop_duplicates(['Vial']).reset_index(drop=True)
          )
maxarea = pd.merge(maxarea,RTmax,how = 'left', on = ['Vial', 'Area'])
maxarea = pd.merge(maxarea,df_vial_counts, how = 'left', on = 'Vial')
maxarea[['Plate','Vial_Well']] = maxarea['Vial'].str.split(':',expand=True)

In [79]:

median_retention_time = maxarea['Retention Time'].median()
min_retention_time = maxarea['Retention Time'].min()
max_retention_time = maxarea['Retention Time'].max()
#print median, min and max retention time
print('Median retention time:', median_retention_time)
print('Minimum retention time:', min_retention_time)
print('Maximum retention time:', max_retention_time)

Median retention time: 4.525
Minimum retention time: 4.504
Maximum retention time: 4.533


calculate RRT values, count of vial peaks, sum of AreaRatio, Assign noise, Shift global RRT option to merge

In [80]:
#calculate relative rention time 'RRT' of all of the peaks
data['RRT'] = data.apply(lambda row: row['Retention Time'] / maxarea[maxarea['Vial'] == row['Vial']]['Retention Time'].values[0], axis=1)
data['RRT'] = data['RRT'].round(2)
#calculate Area ratio of all of the peaks
data['AreaRatio'] = data.apply(lambda row: row['Area']/ maxarea[maxarea['Vial'] == row['Vial']]['Area'].values[0], axis=1)
#count number of rows with each RRT value
df_RRT_counts = data.groupby(['RRT']).size().reset_index(name='peakCount')
#find the RRT value with the highest number of peaks
max_RRT = df_RRT_counts[df_RRT_counts['peakCount'] == df_RRT_counts['peakCount'].max()]['RRT'].values[0]
#return the sum of the area ratio values for each RRT value
df_RRT_area_ratio = data.groupby(['RRT'])['AreaRatio'].sum().reset_index(name='AreaRatio Sum')
df_RRT = pd.merge(df_RRT_counts, df_RRT_area_ratio, on='RRT')
print(df_RRT.head(10))

    RRT  peakCount  AreaRatio Sum
0  0.38          2       0.008530
1  0.96         85       0.464306
2  1.00         84      84.000000
3  1.23          2       0.011536
4  1.25          1       0.010340
5  1.26          4       0.041026
6  1.27         34       0.146039
7  1.28         22       0.042847
8  1.30          2       0.005959
9  1.31          1       0.001705


In [81]:
#subset data to only the rows SampleName, Vial, RRT, and AreaRatio
data = data[['SampleName', 'Vial', 'RRT', 'AreaRatio']]

After you choose which peaks to shift, show what you are shifting in a table essentially

In [82]:
selected_rows_df = pd.DataFrame(columns=data.columns)
df_RRT['shift global RRT'] = 'None'
# Streamlit title
st.title("Select global RRT values for merging peaks")

# Function to find the row with the next least 'RRT' value for the same 'vial'
def find_next_least_rrt(df, vial, rrt):
    subset_df = df[(df['Vial'] == vial) & (df['RRT'] > rrt)]
    if not subset_df.empty:
        next_row = subset_df.loc[subset_df['RRT'].idxmin()]
        return next_row
    else:
        return None

# Function to find the row with the next highest 'RRT' value for the same 'vial'
def find_next_highest_rrt(df, vial, rrt):
    subset_df = df[(df['Vial'] == vial) & (df['RRT'] < rrt)]
    if not subset_df.empty:
        next_row = subset_df.loc[subset_df['RRT'].idxmax()]
        return next_row
    else:
        return None

# Display dropdown menus for each row
st.write("Select an action for each row:")
for index, row in df_RRT.iterrows():
    # Dropdown for selecting an action with 'None' as the default option
    shiftaction = st.selectbox(
        f"Action for {row['shift global RRT']}:",
        options=['None', '< shift to left', '> shift to right'],
        index=0,  # 'None' is the default option
        key=index  # Use the row index as the key for unique identification
    )

    # if 'None' is selected, add the rows from each vial with this RRT value to the 'selected_rows_df'
    if shiftaction == 'None':
        selected_rows_df = selected_rows_df.append(data[data['RRT'] == row['RRT']], ignore_index=True)

    
    # If "< shift to left" is selected, find the next lowest RRT row and combine their arearatios and add to 'selected_rows_df
    if shiftaction == '< shift to left':
        next_row = find_next_least_rrt(data, row['Vial'], row['RRT'])
        if next_row is not None:
            next_row['SampleName'] = row['SampleName']
            next_row['Vial'] = row['Vial']
            next_row['RRT'] = next_row['RRT']
            next_row['AreaRatio'] = row['AreaRatio'] + next_row['AreaRatio']
            selected_rows_df = selected_rows_df.append(next_row, ignore_index=True)
    
    # If "> shift to right" is selected, find the next highest RRT row and combine their arearatios and add to `selected_rows_df`
    elif shiftaction == '> shift to right':
        next_row = find_next_highest_rrt(data, row['Vial'], row['RRT'])
        if next_row is not None:
            next_row['SampleName'] = row['SampleName']
            next_row['Vial'] = row['Vial']
            next_row['RRT'] = next_row['RRT']
            next_row['AreaRatio'] = row['AreaRatio'] + next_row['AreaRatio']
            selected_rows_df = selected_rows_df.append(next_row, ignore_index=True)
# Display the updated DataFrame
pivoted_df = selected_rows_df.pivot_table(index=['SampleName', 'Vial'], columns='RRT', values='AreaRatio', fill_value=0)
st.write("Selected Rows DataFrame:")
st.write(selected_rows_df)
# Convert the DataFrame to a CSV format
csv = pivoted_df.to_csv(index=False)
st.download_button(
    label="Download Output as CSV",
    data=csv,
    file_name='selected_rows.csv',
    mime='text/csv',
)


2024-09-04 11:24:06.685 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-04 11:24:06.687 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-04 11:24:06.688 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-04 11:24:06.689 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-04 11:24:06.690 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-04 11:24:06.692 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-04 11:24:06.694 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-04 11:24:06.695 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

False

In [83]:
print(pivoted_df.head(10))

RRT                      0.38      0.96  1.00  1.23  1.25  1.26      1.27  \
SampleName       Vial                                                       
6 weeks 40C 75RH 3:A,1    0.0  0.005500     1   0.0   0.0   0.0  0.000000   
                 3:A,10   0.0  0.005534     1   0.0   0.0   0.0  0.000000   
                 3:A,11   0.0  0.005632     1   0.0   0.0   0.0  0.000000   
                 3:A,12   0.0  0.005491     1   0.0   0.0   0.0  0.000000   
                 3:A,2    0.0  0.005439     1   0.0   0.0   0.0  0.000000   
                 3:A,3    0.0  0.005483     1   0.0   0.0   0.0  0.000000   
                 3:A,4    0.0  0.005314     1   0.0   0.0   0.0  0.000000   
                 3:A,5    0.0  0.005444     1   0.0   0.0   0.0  0.000000   
                 3:A,6    0.0  0.005423     1   0.0   0.0   0.0  0.000000   
                 3:A,7    0.0  0.005438     1   0.0   0.0   0.0  0.003899   

RRT                      1.28  1.30  1.31  
SampleName       Vial          